<a href="https://colab.research.google.com/github/ThisuriCham/Pytorch/blob/master/custom_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#rescale
class Rescale(object):
  ''' rescale the image in a sample to given size.
  
  Args: 
  output_size(tuple or int): desired output size.
  if tuple, output is matched to output_size. 
  if int, smaller of image edges is matched to 
  output_size keeping aspect ration the same. 
  '''

  def __init__(self, output_size):
    assert isinstance(output_size, (int,tuple))
    self.output_size = output_size

  def __call__(self,sample):
    image, landmarks = sample['image'], sample['landmarks']

    h, w = image.shape[:2]
    if isinstance(self.output_size, int):
       if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        landmarks = landmarks * [new_w / w, new_h / h]

        return {'image': img, 'landmarks': landmarks}




In [0]:
#randomcrop
class RandomCrop(object):
  ''' 
  Crop randomly the image in a sample.

  Args:
  output_size (tuple or int): desired output size. 
  If int, square crop is made.
   '''

   def __init__(self, output_size):
     assert isinstance(output_size, (int, tuple))
     if isinstance(output_size, int):
       self.output_size = (output_size, output_size)
     else:
       assert len(output_size) == 2
       self.output_size = output_size

   def __call__(self,sample):
     image, landmarks = sample['image'],sample['landmarks']
     h, w = image.shape[:2]
     new_h, new_w = self.output_size

     top = np.random.randint(0, h-new_h)
     left = np.random.randint(0, w-new_w)

     image = image[top: top+new_h, left: left+new_w]

     landmarks = landmarks - [left,top]

     return {'image': image, 'landmarks':landmarks}


In [0]:
class ToTensor(object):
  ''' convert ndarrays in sample to tensors '''
  def __call__(self,sample):
    image, landmarks = sample['image'],sample['landmarks']

    #swap color axis
    #numpy image: HxWxC
    #torch image: CxHxW

    image = image.transpose((2,0,1))
    return{'image':torch.from_numpy(image),'landmarks':torch.from_numpy(landmarks)}

**Collate_fn**: this is the function that processes the batch that you want to return from your dataloader.


In [0]:
transformed_dataset = FaceLandmarksDataset(csv_file='data/faces/face_landmarks.csv',root_dir = 'data/faces/',transform=transforms.compose([Rescale(256),RandomCrop(224),ToTensor()]))

dataloader = DataLoader(transformed_dataset, batch_size=4, shuffle=True, num_workers=4)

#helper function to show a batch
def show_landmarks_batch(sample_batched):
  ''' show image with landmarks for a batch os sample '''
   images_batch, landmarks_batch = \
            sample_batched['image'], sample_batched['landmarks']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    grid_border_size = 2

    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

    for i in range(batch_size):
        plt.scatter(landmarks_batch[i, :, 0].numpy() + i * im_size + (i + 1) * grid_border_size,
                    landmarks_batch[i, :, 1].numpy() + grid_border_size,
                    s=10, marker='.', c='r')

        plt.title('Batch from dataloader')

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['landmarks'].size())

    # observe 4th batch and stop.
    if i_batch == 3:
        plt.figure()
        show_landmarks_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break
